In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [64]:
pd.options.mode.chained_assignment = None  # default='warn'
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
db = pd.read_excel('Detections_of_IBC_2018_07_06.xlsx')

In [4]:
def define_db(route):
    db_filtered = db[(db.Route == route)]
    db_filtered.drop(['BorderLocation', 'Route'], axis=1, inplace=True)
    db_filtered.set_index('NationalityLong', inplace=True)

    cluster_now = db_filtered.iloc[:,-2:].sum(axis=1).sort_values(ascending=False)
    cluster_before = db_filtered.iloc[:,-4:-2].sum(axis=1).sort_values(ascending=False)

    top10_now = cluster_now[:10]
    top10_before = cluster_before[:10]

    return top10_now, top10_before

In [109]:
def calculate_results(top10_now, top10_before):
    nationalities = []
    nationalities_dropped = []
    top_num = 0
    
    for nat in top10_now.index:
        if nat in top10_before.index:
            nationality = [nat for nat in top10_before.index if nat == top10_now.index[top_num]]
            result = ((top10_now[nat]-top10_before[nat])/top10_before[nat])*100.0
            nationalities.append([nat, round(result,1)])
        else:
            nationalities.append([nat, 'NEW'])
            top_num += 1
            
    for nat in top10_before.index:
        if nat not in top10_now.index:
            nationalities_dropped.append(nat)
    
    return nationalities, nationalities_dropped

In [110]:
def show_result(result, result_dropped):
    for item in result:
        if isinstance(item[1], float) and not np.isnan(item[1]):
            print(item[0] + ', ' + str(item[1]) + '%')
        else:
            print(item[0] + ', ' + str(item[1]))
    for item in result_dropped:
        print('{} dropped from top 10.'.format(item))

In [111]:
def main(route):
    print('\nResults for {}.\n'.format(route))
    now, before = define_db(route)
    final, final_dropped = calculate_results(now, before)
    show_result(final, final_dropped)

In [112]:
routes = np.unique(db.Route.values)
routes

array(['Black Sea', 'Central Mediterranean',
       'Circular Route from Albania to Greece', 'Eastern Land Borders',
       'Eastern Mediterranean', 'Other', 'Western African',
       'Western Balkans', 'Western Mediterranean'], dtype=object)

In [113]:
for route in routes:
    main(route)


Results for Black Sea.

Morocco, nan
Other and stateless, nan
Turkey, nan
Algeria, nan
Somalia, nan
Iran, nan
Belarus, nan
Afghanistan, nan
Syria, nan
Pakistan, nan

Results for Central Mediterranean.

Tunisia, 172.4%
Sudan, 689.6%
Eritrea, 82.7%
Nigeria, 244.4%
Côte d'Ivoire, 896.7%
Mali, 422.0%
Guinea, 252.0%
Algeria, NEW
Senegal, NEW
Pakistan, NEW
Somalia dropped from top 10.
Niger dropped from top 10.
Iraq dropped from top 10.

Results for Circular Route from Albania to Greece.

Albania, 54.1%
Pakistan, NEW
Kosovo*, 133.3%
China, NEW
Iran, 400.0%
Serbia, NEW
Afghanistan, -40.0%
Brazil, NEW
Turkey, NEW
Morocco, NEW
Iraq dropped from top 10.
Libya dropped from top 10.
Algeria dropped from top 10.
Syria dropped from top 10.
Palestine dropped from top 10.
India dropped from top 10.

Results for Eastern Land Borders.

Vietnam, 1966.7%
Russia, 162.5%
Afghanistan, NEW
Ukraine, 150.0%
Turkey, 11.1%
Iraq, -11.1%
Azerbaijan, NEW
Belarus, NEW
Armenia, NEW
Kyrgyzstan, 0.0%
Cuba dropped from t